In [5]:
from numba import njit, jit
import numpy as np
import time
import scipy
import typing

"""
このファイルは、MaxCut問題に対するQAOA代理アルゴリズムのバージョンを実装しています。
https://journals.aps.org/prresearch/pdf/10.1103/PhysRevResearch.6.023171 を元にしていますが、
分布の直接線形近似を使用しています。
"""

# (start_time, start_value) と (end_time, end_value) の間の直線上で、current_time における y 値を計算
@njit
def line_between(current_time: float, start_time: float, start_value: float, end_time: float, end_value: float) -> float:
    # current_time が start_time から end_time までの間のどこにあるかを計算
    relative_time = (current_time - start_time) / (end_time - start_time)

    # relative_time が 0 から 1 に変化するにつれて、start_value から end_value まで変化。
    return (1 - relative_time) * start_value + relative_time * end_value

# 三角形の曲線上で、与えられた x の位置に対応する y 値を返す
@njit
def triangle_value(x: int, left: int, right: int, height: float) -> float:
    return max(0, min(x - left, right - x) * 2 * height / (right - left))

# コスト c に対する確率 P(c') を計算
@njit
def prob_cost(cost: int, num_constraints: int) -> float:
    return 4 / ((num_constraints + 1) ** 2) * min(cost + 1, num_constraints + 1 - cost)

# コスト c に対する数 N(c') を計算
@njit
def number_with_cost_proxy(cost: int, num_constraints: int, num_qubits: int) -> float:
    scale = 1 << num_qubits
    return prob_cost(cost, num_constraints) * scale

# 距離 d におけるコスト c1 から c2 への遷移の数 N(c'; d, c) を近似的に計算
# TODO: これは prob_edge = 0.5 の場合にのみ機能
@njit
def number_of_costs_at_distance_proxy(cost_1: int, cost_2: int, distance: int, num_constraints: int, num_qubits: int) -> float:
    # 距離が 0 から num_qubits//2 の間にあることを確認
    reflected_distance = distance
    if distance > num_qubits // 2:
        reflected_distance = num_qubits - distance

    # 多項分布のピーク値を近似
    h_peak = 1 << (num_qubits - 4)
    # (0 または num_qubits, 1) と (num_qubits/2, h_peak) の直線上で→
    # reflected_distance におけるピークの高さを計算
    h_at_cost_2 = line_between(reflected_distance, 0, 1, num_qubits / 2, h_peak)
    # cost_1 と num_constraints/2 の直線上で→
    # cost_2 におけるピークの高さを計算
    center = line_between(reflected_distance, 0, cost_1, num_qubits / 2, num_constraints / 2)
    left = center - reflected_distance - 1
    right = center + reflected_distance + 1

    return triangle_value(cost_2, left, right, h_at_cost_2)

# アルゴリズム1のforループ内の和を計算
@njit
def compute_amplitude_sum(prev_amplitudes: np.ndarray, gamma: float, beta: float, cost_1: int, num_constraints: int, num_qubits: int) -> complex:
    sum = 0
    for cost_2 in range(num_constraints + 1):
        for distance in range(num_qubits + 1):
            beta_factor = (np.cos(beta) ** (num_qubits - distance)) * ((-1j * np.sin(beta)) ** distance)
            gamma_factor = np.exp(-1j * gamma * cost_2)
            num_costs_at_distance = number_of_costs_at_distance_proxy(cost_1, cost_2, distance, num_constraints, num_qubits)
            sum += beta_factor * gamma_factor * prev_amplitudes[cost_2] * num_costs_at_distance
    return sum

# 代理の期待値を最適化する代わりに、高コスト振幅を最適化する (例: 指数重み付けを使用して)
# アルゴリズム1を代理の近似を使用して実装
# num_constraints = エッジの数、num_qubits = 頂点の数
@njit
def QAOA_proxy(p: int, gamma: np.ndarray, beta: np.ndarray, num_constraints: int, num_qubits: int):
    num_costs = num_constraints + 1
    amplitude_proxies = np.zeros((p + 1, num_costs), dtype=np.complex128)
    init_amplitude = np.sqrt(1 / (1 << num_qubits))
    for i in range(num_costs):
        amplitude_proxies[0][i] = init_amplitude

    for current_depth in range(1, p + 1):
        for cost_1 in range(num_costs):
            amplitude_proxies[current_depth][cost_1] = compute_amplitude_sum(
                amplitude_proxies[current_depth - 1], gamma[current_depth - 1], beta[current_depth - 1], cost_1, num_constraints, num_qubits
            )

    expected_proxy = 0
    for cost in range(num_costs):
        expected_proxy += number_with_cost_proxy(cost, num_constraints, num_qubits) * (abs(amplitude_proxies[p][cost]) ** 2) * cost

    return amplitude_proxies, expected_proxy

# 逆代理目的関数を作成
def inverse_proxy_objective_function(num_constraints: int, num_qubits: int, p: int, expectations: list[np.ndarray] | None) -> typing.Callable:
    def inverse_objective(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        _, expectation = QAOA_proxy(p, gamma, beta, num_constraints, num_qubits)
        current_time = time.time()

        if expectations is not None:
            expectations.append((current_time, expectation))

        return -expectation

    return inverse_objective

# 現在は prob_edge = 0.5 の場合にのみ実装
def QAOA_proxy_run(
    num_constraints: int,
    num_qubits: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = "COBYLA",
    optimizer_options: dict | None = None,
    expectations: list[np.ndarray] | None = None,
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta])

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_proxy_objective_function(num_constraints, num_qubits, p, expectations), init_freq, args=(), method=optimizer_method, options=optimizer_options
    )
    # 上記は scipy の最適化結果オブジェクトを返す。このオブジェクトには複数の属性がある
    # result.x ：最適解
    # result.success ：アルゴリズムが成功したかどうかを示すブール値
    # result.message ：アルゴリズムが終了した理由のメッセージ
    # result.nfev ：使用された反復回数（ここではQAOA呼び出し回数）
    end_time = time.time()

    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, x = input
        return (time - start_time, x)

    if expectations is not None:
        expectations = list(map(make_time_relative, expectations))

    gamma, beta = result.x[:p], result.x[p:]
    _, expectation = QAOA_proxy(p, gamma, beta, num_constraints, num_qubits)

    return {
        "gamma": gamma,
        "beta": beta,
        "expectation": expectation,
        "runtime": end_time - start_time,  # 秒単位での実行時間
        "num_QAOA_calls": result.nfev,  # 代理のQAOA呼び出し回数
        "classical_opt_success": result.success,
        "scipy_opt_message": result.message,
    }
